In [14]:
from pyensign.events import Event
from pyensign.ensign import Ensign
import pandas as pd
import requests
import json
from sodapy import Socrata
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from cdp_secrets import *

In [15]:
#Constants
chicago_url="data.cityofchicago.org"
crash_api_root="85ca-t3if"

#Chicago Data Portal Connection
cdp = Socrata(chicago_url,
                 token,
                 username,
                 password)

In [16]:
#Get dates for the last 12 months
today = datetime.now().strftime("%Y-%m-%d")

year_ago = (datetime.now() - relativedelta(years=1)).strftime("%Y-%m-%d")
print(today)
print(year_ago)

2023-10-10
2022-10-10


In [17]:
def callAPI(root: str, filter: str) -> pd.DataFrame:
    results=cdp.get(root, query=filter)
    df=pd.DataFrame.from_records(results)
    return df

In [18]:
#limit required for SoQL query, as we are filtering for one year limit is set high to capture all records
limit='1000000000'
crash_filter="""Select crash_record_id,crash_date,beat_of_occurrence as beat_num,crash_month,crash_day_of_week,location
                where crash_date between '%s' and '%s' limit %s"""% (year_ago,today,limit)

In [20]:
#Crash Table
crash_df=callAPI(crash_api_root,crash_filter)

print('completed at ' + str(datetime.now()))

completed at 2023-10-10 17:39:40.126527


In [21]:
crash_df.head()

,crash_record_id,crash_date,beat_num,crash_month,crash_day_of_week,location
0,9f1e2de5e81b78aa98488b4438d5302f4b4d356409868a...,2023-10-09T22:56:00.000,2531,10,2,"{'type': 'Point', 'coordinates': [-87.76648206..."
1,1ef65fe3b4916b9f4ccb4779b2da2b9bea1367c9e987ba...,2023-10-09T21:00:00.000,814,10,2,"{'type': 'Point', 'coordinates': [-87.76230729..."
2,86200eaa8c689f679512b313ce8767da869e8eb84ea2d9...,2023-10-09T20:17:00.000,1133,10,2,"{'type': 'Point', 'coordinates': [-87.71955940..."
3,8ca53301cb1917f37f0dd95c91fed654bc860ceb240b05...,2023-10-09T19:35:00.000,121,10,2,"{'type': 'Point', 'coordinates': [-87.63958775..."
4,a3029f56c4b9d9a4944ff2ffdcb1c631ef90d7212363c8...,2023-10-09T19:30:00.000,2535,10,2,"{'type': 'Point', 'coordinates': [-87.72145164..."


In [23]:
#Convert crash_df to json
crash_json=crash_df.to_json(orient='records')
print(crash_json)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
for crash in crash_json:
    details = crash_json
    if details is None:
        raise Exception(
            "unable to parse usgs api response, no geo-event details found"
        )

    # There's a lot available! For this example, we'll just parse out a few
    # fields from the USGS API response:
    data = {
        "id": details.get("crash_record_id", None),
        "date": details.get("crash_date", None),
        "beat": details.get("beat_num", None),
        "month": details.get("crash_month", None),
        "day": details.get("crash_day_of_week", None),
        "location": details.get("location", None),
    }

    #yield Event(json.dumps(data).encode("utf-8"), mimetype=self.datatype)

AttributeError: 'str' object has no attribute 'get'

In [29]:
client=Ensign(client_id='BHNQsCkeBzrakzwJOUSNTlYqCfebYPGn',client_secret='cqy6VLcAYMz310e9V67NpJAjriw8Y3mwWNLjE0NN3ko7s54Bgfc5TzNjmRJFlLhP')

# Parse the response and publish the event
data = crash_json
event = Event(json.dumps(data).encode("utf-8"))
await client.publish('traffic_incidents', event)
